In [1]:
import pandas as pd
from sp_client import Spotify_Client
from cluster_classifier import predict
import hashlib

In [26]:
sp = Spotify_Client()

In [3]:
recent_df  = pd.read_csv("data/recent.csv")
all_df = pd.read_csv("data/all_songs_clustered.csv", index_col = 'uri')

In [4]:
recent_df.head()

,artist,title,played_at,session,uri,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,label
0,Khalid,OTW,(05/25) 20:12 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6Hgh47WXVKtXN5zGOu0hjI,0.652,0.678,-6.183,0.0541,0.183,0.106,0.28,72.989,4
1,Post Malone,Rich & Sad,(05/25) 19:59 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:2VdT56BGpdqNHUgOe1j5vc,0.599,0.571,-4.998,0.0665,0.198,0.0969,0.285,151.974,4
2,The Chainsmokers,Something Just Like This,(05/25) 19:54 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6RUKPb4LETWmmr3iAEQktW,0.617,0.635,-6.769,0.0317,0.0498,0.164,0.446,103.019,4
3,ILLENIUM,Good Things Fall Apart (with Jon Bellion),(05/25) 19:49 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6pooRNiLyYpxZeIA5kJ5EX,0.643,0.594,-5.453,0.0318,0.0128,0.118,0.442,144.033,4
4,PRETTYMUCH,Phases,(05/25) 19:46 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:3je88Q4OvTqIx7BFRFYvRA,0.77,0.636,-6.724,0.131,0.169,0.0676,0.492,105.93,2


In [10]:
errors = []
for i,r in recent_df.iterrows():
    try:
        recent_df.loc[i, 'danceability'] = float(r['danceability'])
    except ValueError:
        errors.append(i)
        print(i)
        

21
59
60
62
63
64
80
82
83
85
87
88
89
183
194
203
244
302
310


In [16]:
len(errors)

19

In [18]:
list(filter(lambda i: recent_df.loc[i,'uri'] not in all_df.index, errors))

[21,
 59,
 60,
 62,
 63,
 64,
 80,
 82,
 83,
 85,
 87,
 88,
 89,
 183,
 194,
 203,
 244,
 302,
 310]

In [22]:
cols = recent_df.columns[5:-1]

In [23]:
cols

Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'liveness', 'valence', 'tempo'],
      dtype='object')

In [27]:
sp.audio_features('spotify:track:5HBDBBr5OV90lubqW8ctJF')

[{'danceability': 0.958,
  'energy': 0.376,
  'key': 8,
  'loudness': -7.063,
  'mode': 1,
  'speechiness': 0.0523,
  'acousticness': 0.171,
  'instrumentalness': 0,
  'liveness': 0.127,
  'valence': 0.331,
  'tempo': 107.964,
  'type': 'audio_features',
  'id': '5HBDBBr5OV90lubqW8ctJF',
  'uri': 'spotify:track:5HBDBBr5OV90lubqW8ctJF',
  'track_href': 'https://api.spotify.com/v1/tracks/5HBDBBr5OV90lubqW8ctJF',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5HBDBBr5OV90lubqW8ctJF',
  'duration_ms': 144427,
  'time_signature': 4}]

In [32]:
for i in errors:
    uri = recent_df.loc[i, 'uri']
    af = sp.audio_features(uri)[0]
    d = {k:v for (k,v) in af.items() if k in cols}
    recent_df.loc[i, cols] = d
    recent_df.loc[i, 'label'] = predict(list(d.values()))
    
    

In [37]:
cols

Index(['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
       'liveness', 'valence', 'tempo'],
      dtype='object')

In [38]:
for c in list(cols) + ['label']:
    recent_df.loc[:, c] = pd.to_numeric(recent_df.loc[:,c])

In [41]:
recent_df.to_csv("data/recent.csv", index = False)

In [40]:
recent_df.dtypes

artist           object
title            object
played_at        object
session          object
uri              object
danceability    float64
energy          float64
loudness        float64
speechiness     float64
acousticness    float64
liveness        float64
valence         float64
tempo           float64
label             int64
dtype: object

In [34]:
recent_df.loc[:, cols] = pd.to_numeric(recent_df.loc[:, cols])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [33]:
recent_df.loc[errors, ]

,artist,title,played_at,session,uri,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,label
21,Troye Sivan,YOUTH - Gryffin Remix,(05/22) 18:00 PM,4fe4983f67468867df65c465e0b1783e8e822984,spotify:track:5nGuzcsjDbUD1Hgi0szupA,0.559,0.67,-6.301,0.0404,0.0823,0.106,0.338,104.004,4
59,John K,if we never met,(05/19) 17:57 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:5HBDBBr5OV90lubqW8ctJF,0.958,0.376,-7.063,0.0523,0.171,0.127,0.331,107.964,2
60,Greyson Chance,shut up,(05/19) 17:55 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:65mPCrY32YMQlBnmWVrKNa,0.821,0.448,-4.33,0.0468,0.296,0.0683,0.45,105.006,2
62,ayokay,Queen (feat. Quinn XCII),(05/19) 17:48 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:2lxW8vQ9Qjv0qeSQiIBOKJ,0.781,0.619,-6.616,0.0882,0.22,0.112,0.391,82.954,2
63,Quinn XCII,Tough (feat. Noah Kahan),(05/19) 17:45 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:3a1Zmq0P1E6W80x8EOcVNP,0.729,0.57,-7.047,0.0554,0.388,0.111,0.687,93.125,2
64,Chelsea Cutler,Your Shirt,(05/19) 17:42 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:6zdg9ovHEPx9EblJcc7FGf,0.433,0.478,-6.641,0.0725,0.191,0.32,0.173,179.98,3
80,Bazzi,Paradise,(05/17) 18:15 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:0Rx0DJI556Ix5gBny6EWmn,0.844,0.644,-6.273,0.0479,0.0828,0.113,0.591,122.061,4
82,Shoffy,Cool Again,(05/17) 18:09 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:0xM6mdl0XX2qOqUZmVUp97,0.774,0.45,-6.86,0.0896,0.443,0.127,0.648,90.055,2
83,The Band CAMINO,See Through,(05/17) 18:06 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:1pJzl0YsxaWj6BbPs1mUuQ,0.577,0.811,-5.123,0.075,0.0394,0.11,0.666,92.002,0
85,Quinn XCII,Panama,(05/17) 18:00 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:4RtexJZ1OyRkDNlqVUt8Zt,0.73,0.494,-6.402,0.0565,0.57,0.158,0.561,86.732,2


In [31]:

predict(recent_df.loc[errors[0], cols])

4

In [ ]:
#update labels

In [ ]:
af = sp.audio_features(uri)[0]


In [11]:
recent_df.loc[errors, :]

,artist,title,played_at,session,uri,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,label
21,Troye Sivan,YOUTH - Gryffin Remix,(05/22) 18:00 PM,4fe4983f67468867df65c465e0b1783e8e822984,spotify:track:5nGuzcsjDbUD1Hgi0szupA,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav -6...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 0...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 1...,uri\nspotify:track:0fQ9O8i6Hw2VFMW7i1Tgav 4...
59,John K,if we never met,(05/19) 17:57 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:5HBDBBr5OV90lubqW8ctJF,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
60,Greyson Chance,shut up,(05/19) 17:55 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:65mPCrY32YMQlBnmWVrKNa,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
62,ayokay,Queen (feat. Quinn XCII),(05/19) 17:48 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:2lxW8vQ9Qjv0qeSQiIBOKJ,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
63,Quinn XCII,Tough (feat. Noah Kahan),(05/19) 17:45 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:3a1Zmq0P1E6W80x8EOcVNP,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
64,Chelsea Cutler,Your Shirt,(05/19) 17:42 PM,4713ca9a59c874cf161e99d2b334825d1bfd993f,spotify:track:6zdg9ovHEPx9EblJcc7FGf,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
80,Bazzi,Paradise,(05/17) 18:15 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:0Rx0DJI556Ix5gBny6EWmn,"Series([], Name: danceability, dtype: float64)","Series([], Name: energy, dtype: float64)","Series([], Name: loudness, dtype: float64)","Series([], Name: speechiness, dtype: float64)","Series([], Name: acousticness, dtype: float64)","Series([], Name: liveness, dtype: float64)","Series([], Name: valence, dtype: float64)","Series([], Name: tempo, dtype: float64)","Series([], Name: label, dtype: int64)"
82,Shoffy,Cool Again,(05/17) 18:09 PM,8c644c3c80e0885b3470d49280fc59dd25153bae,spotify:track:0xM6mdl0XX2qOqUZmVUp97,"Series([], Name: danceability, dtyp

In [5]:
int('uri')

ValueError: invalid literal for int() with base 10: 'uri'

In [48]:
len({hashlib.sha1(str(x).encode('utf-8')).hexdigest() for x in recent_df.session.unique()})

53

In [49]:
rdf = recent_df.copy()
for s in recent_df.session.unique():
    rdf.loc[recent_df.session == s, 'session'] = hashlib.sha1(str(s).encode('utf-8')).hexdigest()
    

In [62]:
rdf.to_csv("data/recent.csv", index = False)

In [61]:
rdf.head(50)

,artist,title,played_at,session,uri,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,label
0,Khalid,OTW,(05/25) 20:12 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6Hgh47WXVKtXN5zGOu0hjI,0.652,0.678,-6.183,0.0541,0.183,0.106,0.28,72.989,4
1,Post Malone,Rich & Sad,(05/25) 19:59 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:2VdT56BGpdqNHUgOe1j5vc,0.599,0.571,-4.998,0.0665,0.198,0.0969,0.285,151.974,4
2,The Chainsmokers,Something Just Like This,(05/25) 19:54 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6RUKPb4LETWmmr3iAEQktW,0.617,0.635,-6.769,0.0317,0.0498,0.164,0.446,103.019,4
3,ILLENIUM,Good Things Fall Apart (with Jon Bellion),(05/25) 19:49 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:6pooRNiLyYpxZeIA5kJ5EX,0.643,0.594,-5.453,0.0318,0.0128,0.118,0.442,144.033,4
4,PRETTYMUCH,Phases,(05/25) 19:46 PM,d2a04f014f6d12fcb72e387150666fbddfc3dae5,spotify:track:3je88Q4OvTqIx7BFRFYvRA,0.77,0.636,-6.724,0.131,0.169,0.0676,0.492,105.93,2
5,Gryffin,Tie Me Down (with Elley Duh�,(05/24) 12:55 PM,1cec95f1bbeb89273377a16a79df8cc6e3cbc0f9,spotify:track:2zh01m4PiHaZz4BAm0NRxs,0.608,0.845,-2.292,0.0597,0.136,0.111,0.322,98.976,0
6,Joji,TEST DRIVE,(05/24) 11:45 AM,4e60d86009cbb601f461b3996008e3daed6648c2,spotify:track:1DMEzmAoQIikcL52psptQL,0.646,0.648,-8.602,0.0372,0.71,0.108,0.466,75.004,2
7,Joji,Run,(05/24) 11:42 AM,4e60d86009cbb601f461b3996008e3daed6648c2,spotify:track:6p8eEdiZLKJH8tcjGZuNTK,0.447,0.72,-3.954,0.0332,0.0416,0.614,0.362,157.892,3
8,The Chainsmokers,All We Know (feat. Phoebe Ryan),(05/24) 11:39 AM,4e60d86009cbb601f461b3996008e3daed6648c2,spotify:track:2rizacJSyD9S1IQUxUxnsK,0.662,0.586,-8.821,0.0307,0.097,0.115,0.296,90,4
9,Social House,Love Me Back,(05/24) 11:36 AM,4e60d86009cbb601f461b3996008e3daed6648c2,spotify:track:0Vs8BRdNnyuk0QXWPWvICK,0.559,0.437,-10.27,0.0311,0.585,0.101,0.43,157.991,6


In [59]:
int([])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [57]:
rdf.loc[0,'danceability']

'0.652'

In [ ]:
for i,r in rdf.iterrows():
    if type(r['danceability'].value)

In [54]:
try:
    rdf['danceability'] = pd.to_numeric(rdf['danceability']) 
except ValueError:
    # I want to register on my log the message recived on ORIGINAL VALUE
    mask = pd.to_numeric(rdf['danceability'].fillna('0'), errors='coerce').isna() 
    L = rdf.loc[mask, 'danceability'].tolist()
    for val in L:
        print (f"Not converted values are: {val}")

Not converted values are: uri
spotify:track:0fQ9O8i6Hw2VFMW7i1Tgav    0.56
Name: danceability, dtype: float64
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted values are: Series([], Name: danceability, dtype: float64)
Not converted 

In [53]:
rdf.isna().any()

artist          False
title           False
played_at       False
session         False
uri             False
danceability    False
energy          False
loudness        False
speechiness     False
acousticness    False
liveness        False
valence         False
tempo           False
label           False
dtype: bool

In [4]:
all_df.head()

,artist,title,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,pc1,pc2,label
uri,,,,,,,,,,,,,
spotify:track:1iaTDu4PCIWQQOwwwqq5qW,Said the Sky,All I Got,0.325,0.578,-6.786,0.0455,0.158000,0.575,0.103,144.745,-0.165722,0.541033,3
spotify:track:6Q3K9gVUZRMZqZKrXovbM2,Kygo,I'll Wait,0.611,0.651,-6.973,0.0587,0.148000,0.105,0.357,122.983,-0.165114,0.068425,4
spotify:track:75ZKw8JLaFsYr51J44fQ4N,joan,drive all night,0.542,0.758,-5.032,0.0313,0.000184,0.144,0.430,156.128,-0.447970,0.189226,3
spotify:track:6Wz9rIfo9tOBcVCd1Mo7F7,Nightly,Twenty Something,0.641,0.589,-6.305,0.0275,0.096000,0.208,0.110,100.001,-0.125609,0.129021,4
spotify:track:1Srt81FTalOhRw7t7l8Yh8,Lauv,Tattoos Together,0.862,0.400,-7.486,0.0736,0.005750,0.123,0.596,122.963,-0.092945,-0.245872,2


In [30]:
##restructure recent.csv to include audio features + label

# cols =['danceability', 'energy', 'loudness', 'speechiness', 'acousticness',
#        'liveness', 'valence', 'tempo', 'label']
# d = {c:[] for c in cols}
# for i,row in recent_df.iterrows():
#     uri = row['uri']
#     try:
#         af = all_df.loc[uri, cols]
#     except KeyError:
#         try: 
#             af = all_df.loc[(all_df.artist == row.artist) & (all_df.title == row.title), cols]
#         except:
#             print(i)
#     for c in cols:
#         d[c].append(af[c])
# assert len(recent_df) == len(d['danceability'])
# for col in d:
#     recent_df[col] = pd.Series(d[col])
# recent_df.to_csv("data/recent.csv", index = False)

In [ ]:
def get_start_rows(df):
    starts = []
    grouped = df.groupby('session')
    [starts.append(grouped.get_group(k).index.values.astype(int)[0]) for k,_ in grouped]
    return sorted(starts)


In [ ]:
get_start_rows(df)

In [ ]:
df.join(all_df, on = 'uri')

In [ ]:
df